<a href="https://colab.research.google.com/github/grafaelw/Kuhn-PokerBot-cnn-model/blob/main/PokerBot_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! rm -rf data_sets/
! pip install -r requirements-linux.txt
! python data_sets.py

## Defining the training data


In [3]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import files

img_rows, img_cols = 32, 32
batch_size = 32
ROTATE_MAX_ANGLE = 15

dir_path = os.getcwd()
train_data_dir = dir_path + '/data_sets/training_images'

LABELS = ['J','Q','K','A']

features_gen = ImageDataGenerator(rescale=1. / 255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  rotation_range=2*ROTATE_MAX_ANGLE,
                                  horizontal_flip=True,
                                  validation_split=0.2,
                                  fill_mode='nearest')


train_gen = features_gen.flow_from_directory(train_data_dir, color_mode='grayscale',
                                            target_size=(img_rows, img_cols),
                                            batch_size=batch_size,
                                            class_mode='categorical',
                                            subset='training', shuffle=True)

val_gen = features_gen.flow_from_directory(train_data_dir, color_mode='grayscale',
                                          target_size=(img_rows, img_cols),
                                          batch_size=batch_size, 
                                          class_mode='categorical',
                                          subset='validation', shuffle=True)
                                          
train_ds = tf.data.Dataset.from_generator(lambda: train_gen, output_types=(tf.float32,tf.float32), 
                                          output_shapes=([None, img_rows, img_cols, 1],[None, len(LABELS)]))
val_ds = tf.data.Dataset.from_generator(lambda: val_gen, output_types=(tf.float32,tf.float32), 
                                        output_shapes=([None, img_rows, img_cols, 1],[None, len(LABELS)]))




Found 16002 images belonging to 4 classes.
Found 3998 images belonging to 4 classes.


## Defining the model

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, Activation


model = Sequential()
    
# Feature learning 0
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(img_rows, img_cols, 1)))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(img_rows, img_cols, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.1))
    
# Feature learning 1
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.1))

# Fully-connected layer
model.add(Flatten())
model.add(Dense(256, activation='relu',kernel_initializer='he_uniform'))
model.add(Dense(512, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(len(LABELS), activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', 
                  metrics=['accuracy',"mean_squared_error"])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 30, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 28, 28, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 128)       7

## Training the Model

In [6]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
import numpy as np

early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, # Stopping the training if desired validation loss is reached or don't change after 3 epochs
                           verbose=1, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, # Reducing the learning rate if desired validation loss is reached or don't change after 3 epochs
                              verbose=1, min_delta=0.0001, min_lr=0, cooldown=0) 

callbacks = [early_stop, reduce_lr]

model.fit(train_gen, epochs=10, callbacks=callbacks, validation_data=val_gen)
model.save('/model/model.h5', overwrite=True) # Saving the trained model

Epoch 1/20
501/501 [==============================] - 210s 418ms/step - loss: 0.0271 - accuracy: 0.9968 - mean_squared_error: 0.0015 - val_loss: 0.0436 - val_accuracy: 0.9967 - val_mean_squared_error: 0.0015
Epoch 2/20
501/501 [==============================] - 201s 400ms/step - loss: 0.0117 - accuracy: 0.9986 - mean_squared_error: 6.2783e-04 - val_loss: 1.7317e-04 - val_accuracy: 1.0000 - val_mean_squared_error: 2.3190e-05
Epoch 3/20
501/501 [==============================] - 219s 438ms/step - loss: 5.1420e-04 - accuracy: 0.9998 - mean_squared_error: 8.1531e-05 - val_loss: 0.0045 - val_accuracy: 0.9997 - val_mean_squared_error: 1.2506e-04
Epoch 4/20
501/501 [==============================] - 217s 433ms/step - loss: 0.0073 - accuracy: 0.9993 - mean_squared_error: 3.7004e-04 - val_loss: 0.0033 - val_accuracy: 0.9995 - val_mean_squared_error: 1.8946e-04
Epoch 5/20
501/501 [==============================] - 192s 383ms/step - loss: 0.0066 - accuracy: 0.9990 - mean_squared_error: 4.2341e-04

## Evaluating the model with test dataset

At this stage, we have trained our model and it is the time that we should evaluate our trained model whether it is good enough for prediction or not. 
For now, the model is based on a strong notions of VGG-5 neural network architecture which known for its high accuracy and low MSE (Mean Squared Error) value.

In [7]:
test_data_dir = dir_path + '/data_sets/test_images'


testing_gen = ImageDataGenerator(rescale=1. / 255,shear_range=0.2,
                                 zoom_range=0.2, horizontal_flip=True,
                                 rotation_range=2*ROTATE_MAX_ANGLE,
                                 fill_mode='nearest')


test_gen = testing_gen.flow_from_directory(test_data_dir, color_mode='grayscale',
                                            target_size=(img_rows, img_cols),
                                            batch_size=batch_size,
                                            class_mode='categorical',
                                            subset='training', shuffle=True)


                                          
test_ds = tf.data.Dataset.from_generator(lambda: test_gen, output_types=(tf.float32,tf.float32), 
                                          output_shapes=([None, img_rows, img_cols, 1],[None, len(LABELS)]))


score = model.evaluate(test_gen, verbose=1)

print(f"Test Loss = {score[0]*100}%")
print(f"Test Accuracy = {score[1]*100}%")

Found 4000 images belonging to 4 classes.
125/125 [==============================] - 17s 136ms/step - loss: 1.1436e-05 - accuracy: 1.0000 - mean_squared_error: 2.3532e-07
Test Loss = 0.0011435676242399495%
Test Accuracy = 100.0%


In [14]:
from PIL import Image
import numpy as np

def extract_features(image: Image):
    features = np.array(image)
    features = features.reshape(1,features.shape[0], features.shape[1], 1).astype('float32') / 255
    return features


labels = {0:'A', 1:'J', 2:'K', 3:'Q'}
test = [extract_features(Image.open(test_data_dir + '/A/0.png')), # Note: change the file number, because it is randomly generated
        extract_features(Image.open(test_data_dir + '/J/1.png')), # Note: change the file number, because it is randomly generated
        extract_features(Image.open(test_data_dir + '/Q/10.png')), # Note: change the file number, because it is randomly generated
        extract_features(Image.open(test_data_dir + '/K/100.png'))] # Note: change the file number, because it is randomly generated

rank = []

for i in range(len(labels)):
  rank.append(labels[np.argmax(model.predict(test[i]))])
  print(f"Prediction {i+1} = {rank[i]}")



Prediction 1 = A
Prediction 2 = J
Prediction 3 = Q
Prediction 4 = K
